In [3]:
#Los datos usados pueden ser encontrados en https://www.datos.gov.co/Educaci-n/Resultados-nicos-Saber-Pro/u37r-hjmu
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, matthews_corrcoef
from tensorflow import keras
from keras import layers
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier


In [4]:
#Se cargan los datos (solo las columnas de interes) y se toma un numero de filas
#de forma aleatoria

columnas= [ 'ESTU_METODO_PRGM', 'INST_ORIGEN',"ESTU_VALORMATRICULAUNIVERSIDAD",'MOD_INGLES_DESEM',"ESTU_HORASSEMANATRABAJA","ESTU_PAGOMATRICULAPADRES","FAMI_ESTRATOVIVIENDA","FAMI_TIENECOMPUTADOR","FAMI_TIENEINTERNET","ESTU_ESTUDIANTE","ESTU_PAGOMATRICULACREDITO","FAMI_TIENEAUTOMOVIL"]

datos = pd.read_csv("Resultados__unicos_Saber_Pro.csv",usecols=columnas)



#Se eliminan las filas que no tengan valores en las columnas de interes

datos = datos.dropna(subset=columnas)
total_filas = len(datos)


# Se genera una lista de índices aleatorios únicos
indices_aleatorios = random.sample(range(total_filas), 100000)

# Se selecciona las filas correspondientes a los índices aleatorios
filas_aleatorias = datos.iloc[indices_aleatorios]

# Se guardan las etiquetas y las caracteristicas aparte
etiquetas=filas_aleatorias['MOD_INGLES_DESEM']

X = filas_aleatorias.drop("MOD_INGLES_DESEM", axis=1)

**Preparación de las caracteristicas**

In [5]:

#Se trabaja con variables cualitativas, asi que se usa OneHot
one_hot_encoder = OneHotEncoder()
X_encoded = one_hot_encoder.fit_transform(X).toarray()



**PCA**

In [7]:

#Se crea el objeto para aplicar PCA y se aplica a las caracteristicas

pca = PCA(n_components=0.999)
X_pca = pca.fit_transform(X_encoded)


**SVM**

In [12]:
#Se dividen los datos
X_train, X_test, y_train, y_test = train_test_split(X_pca, etiquetas, test_size=0.2, random_state=8)

#Se crea el objeto para aplicar SVM, hiper parametros determinados por prueba y error

model = svm.SVC(C=0.8,degree=3,coef0=4.5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [13]:
f1 = f1_score(y_test, y_pred, average='macro')
print("F1-Score:", f1)

F1-Score: 0.3190983622771792


In [14]:
matthews = matthews_corrcoef(y_test, y_pred)
print("Coeficiente de Matthews:", matthews)

Coeficiente de Matthews: 0.16814128498680472


**Random forest**

In [139]:
# Se dividen los datos
X_train, X_test, y_train, y_test = train_test_split(X_pca, etiquetas, test_size=0.2)

# Se crea el objeto RandomForest y se ajusta a los datos, hiper parametros determinados por prueba y error
forest = RandomForestClassifier(n_estimators=300,criterion='entropy',max_depth=20)

forest.fit(X_train, y_train)

# Se predice a partir de los datos de prueba
y_pred = forest.predict(X_test)

# Se evalua el modelo
f1 = f1_score(y_test, y_pred, average='macro')
mcc = matthews_corrcoef(y_test, y_pred)

print("F1 Score:", f1)
print("MCC:", mcc)

F1 Score: 0.32062465398553536
MCC: 0.15309504502476662


**Red neuronal**

In [9]:
# Se codifican las etiquetas
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(etiquetas)
y_one_hot = keras.utils.to_categorical(y_encoded, num_classes=5)

#Se dividen los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_one_hot, test_size=0.2, random_state=8)

# Se crea el modelo de la red neuronal, cantidad de capas y funciones de activación
#determinadas por prueba y error
model = keras.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(64, activation='softplus'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='softplus'))

model.add(layers.Dense(5, activation='softmax'))

# Se compila el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

# Se entrena el modelo
model.fit(X_train, y_train, epochs=15, batch_size=100, validation_data=(X_test, y_test))

# Se evalua el modelo en el conjunto de prueba
evaluation = model.evaluate(X_test, y_test)
loss = evaluation[0]
accuracy = evaluation[1]


# Se calculan las metricas de interes

y_pred = model.predict(X_test)
mcc = matthews_corrcoef(y_test.argmax(axis=1), y_pred.argmax(axis=1))
f1 = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')


print("Loss:", loss)
print("Accuracy:", accuracy)
print("F1 score:", f1)
print("MCC:", mcc)


Epoch 1/15
800/800 [==============================] - 6s 5ms/step - loss: 1.3788 - precision: 0.5601 - recall: 0.0151 - val_loss: 1.3375 - val_precision: 0.6270 - val_recall: 0.0135
Epoch 2/15
800/800 [==============================] - 3s 4ms/step - loss: 1.3410 - precision: 0.5859 - recall: 0.0205 - val_loss: 1.3361 - val_precision: 0.5694 - val_recall: 0.0225
Epoch 3/15
800/800 [==============================] - 4s 5ms/step - loss: 1.3374 - precision: 0.5838 - recall: 0.0220 - val_loss: 1.3323 - val_precision: 0.6006 - val_recall: 0.0209
Epoch 4/15
800/800 [==============================] - 3s 3ms/step - loss: 1.3356 - precision: 0.5925 - recall: 0.0223 - val_loss: 1.3310 - val_precision: 0.5979 - val_recall: 0.0200
Epoch 5/15
800/800 [==============================] - 3s 4ms/step - loss: 1.3343 - precision: 0.5874 - recall: 0.0228 - val_loss: 1.3314 - val_precision: 0.6802 - val_recall: 0.0117
Epoch 6/15
800/800 [==============================] - 2s 3ms/step - loss: 1.3334 - precisi